In [1]:
import numpy as np

U = lambda θ, ϕ: np.array([
    [0,1j*np.exp(-1j*ϕ),0,0],
    [np.cos(θ), 0, 0, -np.sin(θ)],
    [0,0,1,0],
    [np.sin(θ), 0, 0, np.cos(θ)]
])

In [40]:
import cirq
import numpy as np
circuit = cirq.Circuit()
θ = 0.1
ϕ = 0.5
π = np.pi
q = cirq.LineQubit.range(2)
circuit.append([
            cirq.ZPowGate(exponent=(-1/π) * (ϕ - (π/2))).on(q[1]),
            cirq.X.on(q[0]),
            cirq.CNOT(q[0], q[1]),
            cirq.X.on(q[0]),
            cirq.CXPowGate(exponent=2*θ/π).on(q[1], q[0]),
            cirq.S.on(q[0]),
            cirq.ZPowGate(exponent=-θ/π).on(qubits[1])
])
print(circuit)
print("sin(θ) = ", np.sin(θ))
print("cos(θ) = ",np.cos(θ))
print("i exp(-iϕ) = ",1j * np.exp(-1j*ϕ))
U = cirq.unitary(circuit)
print(np.round(U,3))



0: ───X─────────@───X───X─────────S──────────
                │       │
1: ───Z^0.341───X───────@^0.064───Z^-0.032───
sin(θ) =  0.09983341664682815
cos(θ) =  0.9950041652780258
i exp(-iϕ) =  (0.479425538604203+0.8775825618903728j)


array([[ 0.   +0.j   ,  0.479+0.878j,  0.   +0.j   ,  0.   +0.j   ],
       [ 0.995+0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.088-0.048j],
       [ 0.   +0.j   ,  0.   +0.j   ,  0.   +1.j   ,  0.   +0.j   ],
       [ 0.1  +0.j   ,  0.   +0.j   ,  0.   +0.j   , -0.873+0.477j]])

### Compile right environment unitary

In [19]:
import numpy as np
from scipy.linalg import null_space
def put_env_on_left_site(q, ret_n=False):
    # Take a matrix q (2x2) and create U such that 
    # (right 0-|---|--0
    #          | u |        =  q_{ij}
    # (left) i-|---|--j 
    q = q.T
    a, b, c, d = q.reshape(-1)
    n = np.sqrt(np.abs(a)**2+ np.abs(c)**2+ np.abs(b)**2+ np.abs(d)**2)
    guess = np.array([[a, c.conj(), b, d.conj()], [c, -a.conj(), d, -b.conj()]])/n
    orth = null_space(guess).conj().T
    A = np.concatenate([guess, orth], axis=0)
    SWAP = np.array([
     [1, 0, 0, 0],
     [0, 0, 1, 0],
     [0, 1, 0, 0],
     [0, 0, 0, 1]])
    A = SWAP@A
    if ret_n:
        return A, n
    else:
        return A

    
θ1 = 1
θ2 = 1
ϕ1 = 1
ϕ2 = 1

cos2 = lambda x: np.cos(x)**2
sin2 = lambda x: np.sin(x)**2
e = lambda x: np.exp(1j*x)

num = lambda ϕ2: 2*e(ϕ2)
den = lambda ϕ1,ϕ2,θ1,θ2: np.sqrt((e(ϕ1)*cos2(θ2)*cos2(θ1) + 4*e(ϕ2)*np.sin(θ1)*np.sin(θ2)) * e(ϕ1)) + e(ϕ1)*np.cos(θ1)*np.cos(θ2)

R00 = lambda ϕ1,ϕ2,θ1,θ2: -num(ϕ2) / den(ϕ1,ϕ2,θ1,θ2)

R10 = lambda θ: 1 / np.tan(θ)

R = lambda ϕ1,ϕ2,θ1,θ2: np.array([
    [R00(ϕ1,ϕ2,θ1,θ2), R10(θ1)],
    [R10(θ2), 1]
])

UR = lambda ϕ1,ϕ2,θ1,θ2: put_env_on_left_site(R(ϕ1,ϕ2,θ1,θ2))

print(np.linalg.norm(R(1,1,1,1)))



1.6806444760492278


### Compile Many Body Scars

In [28]:
# This is the original circuit we define

import cirq
import numpy as n
from sympy import Symbol
circuit = cirq.Circuit()
θ = 0.1#, Symbol('θ')
ϕ = 0.1#, Symbol('ϕ')
π = np.pi#,Symbol('π')
q = cirq.LineQubit.range(2)
circuit.append([
            cirq.ZPowGate(exponent=(-1/π) * (ϕ - (π/2))).on(q[1]),
            cirq.X.on(q[0]),
            cirq.CNOT(q[0], q[1]),
            cirq.X.on(q[0]),
            cirq.CXPowGate(exponent=2*θ/π).on(q[1], q[0]),
            cirq.S.on(q[0]),
            cirq.ZPowGate(exponent=-θ/π).on(q[1])
])
print(circuit)



0: ───X─────────@───X───X─────────S──────────
                │       │
1: ───Z^0.468───X───────@^0.064───Z^-0.032───


In [19]:
# Thing to try:

# ConvertToCZAndSingleGates
# EjectPhasedPaulis
# EjectZ
# MergeSinlgeQubitGates
# optimized_for_sycamore

cirq.ConvertToCzAndSingleGates().optimize_circuit(circuit = circuit)
for moment in circuit:
    print(moment)

Z**0.4681690113816209(1) and X(0)
PhX(-0.5)^0.5(1)
CZ**-1.0(0, 1)
X(0) and PhX(0.5)^0.5(1)
PhX(-0.518)^0.5000000000000036(1) and PhX(0.5)^0.499999999999992(0)
CZ(1, 0)
PhX(0.982)^0.031830988618378964(1)
CZ(1, 0)
PhX(-0.468)^0.5(0) and PhX(0.482)^0.5000000000000036(1)
Z**-0.0318309886183783(0) and Z**0.03183098861837963(1)
S(0) and Z**-0.03183098861837907(1)


In [21]:
cirq.EjectZ().optimize_circuit(circuit=circuit)
for moment in circuit:
    print(moment)

X(0)
PhX(-0.968)^0.5(1)
CZ**-1.0(0, 1)
X(0) and PhX(0.0318)^0.5(1)
PhX(-0.986)^0.5000000000000036(1) and PhX(0.5)^0.499999999999992(0)
CZ(1, 0)
PhX(0.514)^0.031830988618378964(1)
CZ(1, 0)
PhX(-0.468)^0.5(0) and PhX(0.0139)^0.5000000000000036(1)
Z**0.4681690113816217(0) and Z**0.4681690113816215(1)



In [22]:
cirq.EjectPhasedPaulis().optimize_circuit(circuit=circuit)
for moment in circuit:
    print(moment)


PhX(-0.968)^0.5(1)
Z**-1.0(1)
CZ(0, 1)
PhX(0.0318)^0.5(1)
PhX(-0.986)^0.5000000000000036(1) and PhX(0.5)^0.499999999999992(0)
CZ(1, 0)
PhX(0.514)^0.031830988618378964(1)
CZ(1, 0)
PhX(-0.468)^0.5(0) and PhX(0.0139)^0.5000000000000036(1)
Z**0.4681690113816217(0) and Z**0.4681690113816215(1)



In [29]:
for m in cirq.google.optimized_for_sycamore(circuit = circuit, optimizer_type='sycamore'):
    print(m)

PhXZ(a=0.0318,x=0.5,z=0)(1) and PhXZ(a=-0.5,x=0.414,z=0)(0)
SYC(0, 1)
PhXZ(a=0.417,x=0.477,z=0)(1)
SYC(0, 1)
PhXZ(a=0.833,x=0.5,z=0)(0) and PhXZ(a=0.861,x=0.415,z=0)(1)
SYC(1, 0)
PhXZ(a=0.279,x=0.967,z=0)(0)
SYC(1, 0)
PhXZ(a=0.195,x=0.504,z=0.72)(1) and PhXZ(a=1.28,x=0.5,z=-1.28)(0)


In [30]:
len(cirq.google.optimized_for_sycamore(circuit = circuit, optimizer_type='sycamore'))


9